In [ ]:
### Convolution Neural Network 합성곱 신경망
import numpy as np
import pandas as pd 
from sklearn.neighbors import KNeighborsRegressor
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from scipy import stats

from PIL import Image

print('CNN')

# 이미지 소환
img = Image.open('/content/drive/My Drive/MachineLearning/data/img/img1020.jpg')
plt.imshow(img)
plt.show()

# pixel데이터를 ndarray로 뽑자
pixel = np.array(img)
print("이미지 사이즈 : ", img.size)

# 특정 위치의 pixel값을 알아보기
print('X : {}, Y : {} 의 픽셀값은 : {}'.format(100,200, pixel[200,100]))
# 여기서는 앞에가 세로 뒤에가 가로임 ㄷㄷ

# 2. 이미지 크롭
crop_image = img.crop((1000,3000,2500,4500))
plt.imshow(crop_image)
plt.show()

# 3. 리사이즈
resize_img = img.resize((int(img.size[0]/4), int(img.size[1]/4)))
plt.imshow(resize_img)
plt.show()

# 4. 이미지회전
rotate_img = img.rotate(180)
plt.imshow(rotate_img)
plt.show()

# 5. 흑백처리
grey_pixel = pixel.copy()
for y in range(grey_pixel.shape[0]):
    for x in range(grey_pixel.shape[1]):
        grey = int((grey_pixel[y,x,0] + grey_pixel[y,x,1] + grey_pixel[y,x,2])/3)
        grey_pixel[y,x,0] = grey
        grey_pixel[y,x,1] = grey
        grey_pixel[y,x,2] = grey
        
        
# plt.imshow(grey_pixel)
# plt.show()  

print(grey_pixel.shape)
print(grey_pixel[:,:,0:1].shape)

plt.imshow(grey_pixel[:,:,0], cmap='Greys')
plt.show()

In [ ]:
import numpy as np
import tensorflow as tf

# 입력이미지
# (1, 4, 4, 1) => (이미지개수, height, width, channel)
image = np.array([[[[1],[1],[1],[1]],
                 [[1],[1],[1],[1]],
                 [[1],[1],[1],[1]],
                 [[1],[1],[1],[1]]]], dtype=np.float32)
print(image.shape)

# pooling??
pool = tf.nn.max_pool(image, 
                      ksize=[1,2,2,1],
                      strides=[1,2,2,1],
                      padding='VALID')

sess = tf.Session()
result = sess.run(pool)

print(result.shape)

In [ ]:
!pip uninstall tensorflow

In [ ]:

!pip install tensorflow==1.15

In [ ]:
%reset
### CNN mnist in tensorflow 1.15
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
import tensorflow as tf
import matplotlib.pyplot as plt

print(tf.__version__)

## raw data loading
df = pd.read_csv('/content/drive/My Drive/MachineLearning/data/mnist/train.csv')
print(df.shape)

# 결측치 이상치 x

## data split
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df.drop('label', axis=1, inplace=False), 
                 df['label'], test_size=0.3, random_state=0)

## nomalization
scaler = MinMaxScaler()
scaler.fit(x_data_train)
x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)
del x_data_train, x_data_test

## tensorflow implemention
sess = tf.Session()
t_data_train_onehot = sess.run(tf.one_hot(t_data_train, depth=10))
t_data_test_onehot = sess.run(tf.one_hot(t_data_test, depth=10))

## Placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
T = tf.placeholder(shape=[None,10], dtype=tf.float32)
drop_rate = tf.placeholder(dtype=tf.float32)

## Convolution Layer
# 입력데이터 형태부터 설정
x_img = tf.reshape(X, [-1, 28, 28, 1])

# convolution layer1
W1 = tf.Variable(tf.random.normal([3,3,1,32]))  # height width channel 필터개수
L1 = tf.nn.conv2d(x_img,W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)
# pooling layer1
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
print(L1.shape)

# convolution layer2
W2 = tf.Variable(tf.random.normal([3,3,32,64]))  # height width channel 필터개수
L2 = tf.nn.conv2d(L1,W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)
# pooling layer2
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
print(L2.shape)

## FC에 넣어서 학습을 진행
L2 = tf.reshape(L2, [-1,7*7*64])

# weight & bias
W3 = tf.get_variable('weight3', shape=[7*7*64, 256], 
                     initializer=tf.contrib.layers.variance_scaling_initializer())
b3 = tf.Variable(tf.random.normal([256]))

_layer3 = tf.nn.relu(tf.matmul(L2,W3) + b3)
layer3 = tf.nn.dropout(_layer3, rate=drop_rate)

W4 = tf.get_variable('weight4', shape=[256, 10], 
                     initializer=tf.contrib.layers.variance_scaling_initializer())
b4 = tf.Variable(tf.random.normal([10]))

# Hypothesis
logit = tf.matmul(layer3,W4) + b4
H = tf.nn.softmax(logit)

# loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, 
                                                                 labels=T))

# train
train = tf.train.AdadeltaOptimizer(learning_rate=1e-3).minimize(loss)


num_of_epoch = 200
batch_size = 100
# 학습
def run_train(sess, train_x, train_t):
    print('### Starting Training ###')
    # 초기화
    sess.run(tf.global_variables_initializer())
    
    for step in range(num_of_epoch):
        total_batch = int(train_x.shape[0] / batch_size)

        for i in range(total_batch):
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_t = train_t[i*batch_size:(i+1)*batch_size]
            _, loss_val = sess.run([train,loss], feed_dict={X:batch_x, T:batch_t, drop_rate:0.4})

        if step % 20 == 0:
            print('Loss : {}'.format(loss_val))
    print('### End Training ###')

    
# Accuracy
predict = tf.argmax(H,1)

# Testing
# sklearn의 classification_report를 이용한 성능평가
run_train(sess,x_data_train_norm,t_data_train_onehot)
target_names=['num 0','num 1','num 2','num 3','num 4','num 5','num 6','num 7','num 8','num 9']
print(classification_report(t_data_test,
                            sess.run(predict,
                                     feed_dict={X:x_data_test_norm, 
                                                drop_rate:0 }),
                            target_names=target_names))  